### Instructions
I would recommend running this program on the command line.
use: 'python3 02-project.py'

In [6]:
# Tyler Percy
# 2/19/21
# PHY 2200

from vpython import *
from numpy import zeros

class Bullet:

    mass = 1e4
    body: sphere
    g = vec(0, -.1, 0)
    Fgrav = g*mass
    velocity: vec
    air_resist = 200

    def __init__(self, pos, vel):
        self.body = sphere(pos = pos, radius = .5, make_trail=True)
        self.velocity = vel

    def get_diffeq(self, d, tn):
        speed = mag(vec(d[3], d[4], d[5]))

        rates = zeros(6)
        rates[0] = d[3]
        rates[1] = d[4]
        rates[2] = d[5]
        rates[3] = (self.Fgrav.x-self.air_resist*speed*d[3])/self.mass
        rates[4] = (self.Fgrav.y-self.air_resist*speed*d[4])/self.mass
        rates[5] = (self.Fgrav.z-self.air_resist*speed*d[5])/self.mass
        return rates


In [7]:
# Tyler Percy
# 2/12/21
# PHY 2200

from vpython import *
from bullet import Bullet
from ode import RK4
from numpy import array

class Tank:

    body: box
    cannon: cylinder
    pos: vec
    color: color

    const_v = .05
    axis_rotate_scale = .02
    
    current_bullet: sphere
    bullet_v = .5
    hit_target = False

    def __init__(self, xpos, ypos, zpos, xaxis, color):
        self.body = box(pos = vec(xpos, ypos, zpos), size = vec(2,2,.5), color = color)
        self.cannon = cylinder(pos = vec(xpos, ypos+.75, zpos), color = color, axis = vec(xaxis,1,0)*2, radius = .5)
        self.color = color

    def move(self):
        k = keysdown()
        if 'left' in k:
            self.body.pos.x -= self.const_v
            self.cannon.pos.x -= self.const_v
        if 'right' in k:
            if self.cannon.pos.x < -3.5 or self.cannon.pos.x > 3.5:
                self.body.pos.x += self.const_v
                self.cannon.pos.x += self.const_v
        if 'up' in k:
            if self.color == color.red:
                if self.cannon.axis.x > 1:
                    self.cannon.axis.y += self.axis_rotate_scale
                    self.cannon.axis.x -= self.axis_rotate_scale
            else:
                if self.cannon.axis.x < -1:
                    self.cannon.axis.y += self.axis_rotate_scale
                    self.cannon.axis.x += self.axis_rotate_scale
        if 'down' in k:
            if self.color == color.red:
                if self.cannon.axis.x < 3:
                    self.cannon.axis.y -= self.axis_rotate_scale
                    self.cannon.axis.x += self.axis_rotate_scale
            else:
                if self.cannon.axis.x > -3:
                    self.cannon.axis.y -= self.axis_rotate_scale
                    self.cannon.axis.x -= self.axis_rotate_scale

        if 'w' in k:
            if -.005 <= self.bullet_v < 1:
                self.bullet_v += .005
        if 's' in k:
            if 0 < self.bullet_v <= 1.005:
                self.bullet_v -= .005

    def fire(self, target, t, dt):

        if self.color == color.red:
            self.current_bullet = Bullet(pos=vec(self.cannon.pos.x+.5, self.cannon.pos.y+.5, self.cannon.pos.z), \
                    vel=vec(self.bullet_v*self.cannon.axis.x, self.bullet_v*self.cannon.axis.y, 0))
        else:
            self.current_bullet = Bullet(pos=vec(self.cannon.pos.x, self.cannon.pos.y, self.cannon.pos.z), \
                    vel=vec(self.bullet_v*self.cannon.axis.x, self.bullet_v*self.cannon.axis.y, 0))

        data = array([self.current_bullet.body.pos.x, self.current_bullet.body.pos.y, self.current_bullet.body.pos.z, \
            self.current_bullet.velocity.x, self.current_bullet.velocity.y, self.current_bullet.velocity.z])

        while (self.current_bullet.body.pos.y > -13.5):
            if (-1 < self.current_bullet.body.pos.x < 1) and (-14 < self.current_bullet.body.pos.y < -2):
                print("Hit wall")
                break
            
            # if bullet collides with target
            if target.body.pos.x-1 < self.current_bullet.body.pos.x < target.body.pos.x+1 and \
                target.body.pos.y-1 < self.current_bullet.body.pos.y < target.body.pos.y+1:
                    print("Target hit")
                    self.hit_target = True
                    break
            
            data = RK4(self.current_bullet.get_diffeq, data, t, dt)
            self.current_bullet.body.pos = vec(data[0], data[1], data[2])
            self.current_bullet.velocity = vec(data[3], data[4], data[5])

        self.current_bullet.body.clear_trail()
        self.current_bullet.body.visible = False
    

In [8]:
# Tyler Percy
# 2/12/21
# PHY 2200

from vpython import *
from interface import Interface
from random import randint, uniform

class World:

    def __init__(self) -> None:
        #terrain and background
        self.scene = canvas(title = "Tank Wars", width = 1600, height = 900, range = 20, background = vec(.1,.1,.1))
        self.floor = box(pos = vec(0, -15, 0), size = vec(60, 2.5, .5), color = vec(.5,.5,.5))
        self.wall = box(pos = vec(0,-8.25,0), size = vec(1, 12, .5), color = vec(.5,.5,.5))
        self.sun = sphere(pos = vec(0,-15,-.5), size = vec(50,50,.01), color = vec(.9,.6,.2))
        self.UI = Interface()

        #stars
        for _ in range(0, 100):
            box(pos = vec(randint(-30, 30), randint(-15,15), -.6), size = vec(uniform(.15,.25), uniform(.15,.25), .05), color = color.white)

        #lock camera
        self.scene.resizable = False
        self.scene.userzoom = False
        #refocus camera
        self.scene.fov = .1



In [9]:
# Tyler Percy
# 2/12/21
# PHY 2200

from vpython import *
from tank import Tank

class Interface:

    left_axis_tilt: cylinder
    left_projectile_velocity: cylinder

    left_tilt_label: label
    right_tilt_label: label
    left_vel_label: label
    right_vel_label: label

    right_axis_tilt: cylinder
    left_projectile_velocity: cylinder

    def __init__(self):
        self.left_axis_tilt = cylinder(pos = vec(-20,-17.5,0), axis = vec(1,0,0)*4, color = vec(.8,0,0), radius = .5)
        self.left_projectile_velocity = cylinder(pos = vec(-20,-19,0), axis = vec(1,0,0)*4, color = vec(1,.5,.5), radius = .5)

        self.left_tilt_label = label(pos = vec(-22.5, -17.5, 0), text = "AXIS TILT")
        self.left_vel_label = label(pos = vec(-22.5, -19, 0), text = "VELOCITY")

        self.right_tilt_label = label(pos = vec(22.5, -17.5, 0), text = "AXIS TILT")
        self.right_vel_label = label(pos = vec(22.5, -19, 0), text = "VELOCITY")

        self.right_axis_tilt = cylinder(pos = vec(20,-17.5,0), axis = vec(-1,0,0)*4, color = color.blue, radius = .5)
        self.right_projectile_velocity = cylinder(pos = vec(20,-19,0), axis = vec(-1,0,0)*4, color = color.cyan, radius = .5)

    def adjust_axis_display(self, tank):
        if tank.color == color.red:
            self.left_axis_tilt.axis.x = (3-tank.cannon.axis.x)*3
        else:
            self.right_axis_tilt.axis.x = -(tank.cannon.axis.x+3)*3

    def adjust_velocity_display(self, tank):
        if tank.color == color.red:
            self.left_projectile_velocity.axis.x = tank.bullet_v*6
        else:
            self.right_projectile_velocity.axis.x = -tank.bullet_v*6
    

In [10]:
# Tyler Percy
# 2/1/21
# PHY 2200

def Euler(diffeq, yn, tn, h):
    """ Given y_n at t, calculate and return y_n+1 at t+h """

    #calculate y_n+1
    yn1 = yn + diffeq(yn,tn)*h

    return yn1

def RK2(diffeq, yn, tn, h):
    k1 = h*diffeq(yn, tn)
    k2 = h*diffeq(yn+(k1/2), tn+(h/2))
    
    yn1 = yn + k2
    
    return yn1

def RK4(diffeq, yn, tn, h):
    k1 = h*diffeq(yn, tn)
    k2 = h*diffeq(yn+(k1/2), tn+(h/2))
    k3 = h*diffeq(yn+(k2/2), tn+(h/2))
    k4 = h*diffeq(yn+k3, tn+h)
    
    yn1 = yn + k4
    
    return yn1


In [ ]:
# Tyler Percy
# 2/12/21
# PHY 2200

from world import World
from tank import Tank
from vpython import *

def main():
    t = 0
    dt = .0005

    map = World()

    tank1 = Tank(xpos=-20, ypos=-12.75, zpos=.5, xaxis = 1, color=color.red)
    tank2 = Tank(xpos=20, ypos=-12.75, zpos=.5, xaxis = -1, color=color.cyan)

    # red tank goes first
    control_tank = tank1
    enemy_tank = tank2

    while(True):
        rate(100)
        control_tank.move()
        if 'shift' in keysdown():
            control_tank.fire(enemy_tank, t, dt)

            if (control_tank.hit_target):
                enemy_tank.body.visible = False
                enemy_tank.cannon.visible = False
                break
            else:
                #swap control to other tank
                temp = control_tank
                control_tank = enemy_tank
                enemy_tank = temp

        map.UI.adjust_axis_display(control_tank)
        map.UI.adjust_velocity_display(control_tank)

        t = t + dt
    
if __name__ == "__main__":
    main()

### Narrative

The purpose of this program is to demonstrate the ODE solver by creating a 'tank wars' variant.  The model consists of two tanks on opposite ends that take turns firing at eachother until the bullet from one tank hits the other.  The ODE solver comes into play when updating the velocity and position of the bullet.  During the course of the game, each tank is allowed to move, increase/decrease the angle of their cannon, and increase/decrease the initial velocity of the bullet.

Another component of this program is air resistance.  This value can be adjusted in bullet.py and has a large effect on the curvature of the bullet while it is in the air.  Generally speaking, a higher value makes the overall curvature of the bullet less parabolic.  The addition of air resistance can make the game more difficult as it is harder to simply visualize a perfect curvature and add more realism, as well as more challenge.

I believe the program is working as intended because by using the RK4 solver, there is an accurate representation of the bullet's trajectory.  The air resistance also appears to be modeled accurately as demonstrated by increasing or decreasing it and observing the results.

